In [2]:
import Arena
from MCTS import MCTS

from antichess3.AntiChessGame import  AntiChessGame
from antichess3.AntiChessPlayers import *


from antichess3.pytorch.NNet import NNetWrapper as NNet
import os


import numpy as np
from utils import *

import time

"""
use this script to play any two agents against each other, or play manually with
any agent.
"""

g = AntiChessGame(8)
#g = TicTacToeGame()
#g = ChessGame()

# all players
rp1 = RandomPlayer(g).play
rp2 = RandomPlayer(g).play

stockfish1 = StockFishPlayer(g,3000).play
stockfish2 = StockFishPlayer(g,500).play


#hp = HumanPawnsPlayer(g).play

# nnet players
n1 = NNet(g)
n1.load_checkpoint('./temp/antichess/final/torch/200x100x35x0.6x200000x0.1x150x1.25/','best.pth.tar')

args1 = dotdict({'numMCTSSims': 150, 'cpuct':1.25, 'dirichletAlpha' : 0.1})
mcts1 = MCTS(g, n1, args1, dirichlet_noise=True)
n1p = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))

n2 = NNet(g)
n2.load_checkpoint('./temp/antichess/final/torch/200x100x35x0.6x200000x0.1x150x1.25/','checkpoint_1.pth.tar')

args2 = dotdict({'numMCTSSims': 150, 'cpuct':1.25, 'dirichletAlpha' : 0.1})
mcts2 = MCTS(g, n2, args2, dirichlet_noise=True)
n2p = lambda x: np.argmax(mcts2.getActionProb(x, temp=0))

arena = Arena.Arena(n1p,n2p,g)


start = time.time()
print(arena.playGames(40,verbose=False))
end = time.time()
print(end-start)



TypeError: exceptions must derive from BaseException

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.enabled)
print()

torch.zeros(5).cuda()

True
11.1
True



tensor([0., 0., 0., 0., 0.], device='cuda:0')

In [4]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti'

In [5]:
n1 = NNet(g)
n1.load_checkpoint('./temp/antichess/reduced_moves/torch/200x100x25x25x1/','checkpoint_24.pth.tar')

args1 = dotdict({'numMCTSSims': 100, 'cpuct':1.25, 'dirichletAlpha' : 0.1})
mcts1 = MCTS(g, n1, args1, dirichlet_noise=True)
n1p = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))

TypeError: exceptions must derive from BaseException

In [6]:
arena = Arena(n1p,rp1,g)


start = time.time()
print(arena.playGame(verbose=False))
end = time.time()
print(end-start)


NameError: name 'n1p' is not defined

In [7]:
import numpy as np
#from pytorch_classification.utils import Bar, AverageMeter
from utils import *
from antichess3.progress.bar import Bar
import time


class Arena():
    """
    An Arena class where any 2 agents can be pit against each other.
    """

    def __init__(self, player1, player2, game, display=None):
        """
        Input:
            player 1,2: two functions that takes board as input, return action
            game: Game object
            display: a function that takes board as input and prints it (e.g.
                     display in othello/OthelloGame). Is necessary for verbose
                     mode.
        see othello/OthelloPlayers.py for an example. See pit.py for pitting
        human players/other baselines with each other.
        """
        self.player1 = player1
        self.player2 = player2
        self.game = game
        self.display = display
        self.displaybar = True

    def playGame(self, verbose=False):
        """
        Executes one episode of a game.
        Returns:
            either
                winner: player who won the game (1 if player1, -1 if player2)
            or
                draw result returned from the game that is neither 1, -1, nor 0.
        """
        players = [self.player2, None, self.player1]
        curPlayer = 1
        board = self.game.getInitBoard()
        it = 0
        while self.game.getGameEnded(board, curPlayer) == 0:
            it += 1
            if verbose:
                assert (self.display)
                print("Turn ", str(it), "Player ", str(curPlayer))
                self.display(board)
            action = players[curPlayer + 1](self.game.getCanonicalForm(board, curPlayer))

            valids = self.game.getValidMoves(self.game.getCanonicalForm(board, curPlayer), 1)

            if valids[action] == 0:
                print(action)
                assert valids[action] > 0
            board, curPlayer = self.game.getNextState(board, curPlayer, action)
        if verbose:
            assert (self.display)
            print("Game over: Turn ", str(it), "Result ", str(self.game.getGameEnded(board, 1)))
            self.display(board)
        return self.game.getGameEnded(board, 1)

    def playGames(self, num, verbose=False):
        """
        Plays num games in which player1 starts num/2 games and player2 starts
        num/2 games.
        Returns:
            oneWon: games won by player1
            twoWon: games won by player2
            draws:  games won by nobody
        """
        eps_time = AverageMeter()
        bar = Bar('Arena.playGames', max=num)
        end = time.time()
        eps = 0
        maxeps = int(num)

        num = int(num / 2)
        oneWon = 0
        twoWon = 0
        draws = 0
        for _ in range(num):
            gameResult = self.playGame(verbose=verbose)
            if gameResult == 1:
                oneWon += 1
            elif gameResult == -1:
                twoWon += 1
            else:
                draws += 1
            # bookkeeping + plot progress
            eps += 1
            eps_time.update(time.time() - end)
            end = time.time()
            if (self.displaybar):
                bar.suffix = '({eps}/{maxeps}) Eps Time: {et:.3f}s | Total: {total:} | ETA: {eta:}'.format(eps=eps + 1,
                                                                                                           maxeps=maxeps,
                                                                                                           et=eps_time.avg,
                                                                                                           total=bar.elapsed_td,
                                                                                                           eta=bar.eta_td)
                bar.next()

        self.player1, self.player2 = self.player2, self.player1

        for _ in range(num):
            gameResult = self.playGame(verbose=verbose)
            if gameResult == -1:
                oneWon += 1
            elif gameResult == 1:
                twoWon += 1
            else:
                draws += 1
            # bookkeeping + plot progress
            eps += 1
            eps_time.update(time.time() - end)
            end = time.time()
            if (self.displaybar):
                bar.suffix = '({eps}/{maxeps}) Eps Time: {et:.3f}s | Total: {total:} | ETA: {eta:}'.format(eps=eps + 1,
                                                                                                           maxeps=num,
                                                                                                           et=eps_time.avg,
                                                                                                           total=bar.elapsed_td,
                                                                                                           eta=bar.eta_td)
                bar.next()

        bar.finish()

        return oneWon, twoWon, draws

In [8]:
models = os.listdir("C:\\Users\\Jernejc\\PycharmProjects\\MasterThesis\\temp\\antichess\\reduced_moves\\torch\\200x100x25x0.6x200000x0.1x100x1.25")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Jernejc\\PycharmProjects\\MasterThesis\\temp\\antichess\\reduced_moves\\torch\\200x100x25x0.6x200000x0.1x100x1.25'

In [9]:
results = []

In [10]:
results_stock = []

In [11]:
def myFunc(e):
    try:
        model_index = int(e.split(".")[0].split("_")[-1])
    except:
        model_index = 201
    return model_index

In [12]:
myFunc('checkpoint_100.pth.tar')

100

In [13]:
models = os.listdir("C:\\Users\\Jernejc\\PycharmProjects\\MasterThesis\\temp\\antichess\\reduced_moves\\torch\\200x100x25x0.6x200000x0.1x100x1.25")
models.sort(key=myFunc)
models

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Jernejc\\PycharmProjects\\MasterThesis\\temp\\antichess\\reduced_moves\\torch\\200x100x25x0.6x200000x0.1x100x1.25'

In [ ]:
models

In [14]:
for model in models[82:]:
    if 'checkpoint' in model:
        print(model)
        rp1 = RandomPlayer(g).play
        
        n1 = NNet(g)
        n1.load_checkpoint('./temp/antichess/reduced_moves/torch/200x100x25x0.6x200000x0.1x100x1.25/',model)
        
        #stockfish1 = StockFishPlayer(g,1000).play


        args1 = dotdict({'numMCTSSims': 100, 'cpuct':1.25, 'dirichletAlpha' : 0.1})
        mcts1 = MCTS(g, n1, args1, dirichlet_noise=True)
        n1p = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))
        
        arena = Arena.Arena(n1p,rp1, g)
        
        start = time.time()
        one,two,draw = arena.playGames(20,verbose=False)
        print(one,two,draw,model)
        results_stock.append((one,two,draw,model))
        end = time.time()
        print(end-start)


NameError: name 'models' is not defined

In [ ]:
import numpy as np

In [ ]:
pi = [0.0,0.0,0.1,0.0,0.0,0.01,0.49,0.0,0.15,0.0,0.05,0.0,0.06,0.06,0.04,0.04]

In [ ]:
if not url in urls_d:
        urls_d[url] = 1
    else:
        urls_d[url] += 1

In [ ]:
res = {}

In [ ]:
for _ in range(1000):
    action = np.random.choice(len(pi), p=pi)
    if not action in res:
        res[action] = 1
    else:
        res[action] += 1

In [ ]:
res

In [ ]:
def get_move_evaluation(engine, board, time_per_move=0.01):
        list_moves = []
        for el in board.legal_moves:
            info = engine.analyse(board, chess.engine.Limit(time=time_per_move), root_moves=[el])
            t = str(info["score"])
            # print(el,info["score"].pov(board.turn).score(mate_score=1000000),t)
            list_moves.append((str(el), info["score"].pov(board.turn).score(mate_score=1000000)))

        sorted_list_moves = sorted(list_moves, key=lambda x: x[1], reverse=True)
        # for move in sorted_list_moves:
        #    print(move)
        return dict(sorted_list_moves)

In [ ]:
import chess
import chess.variant
import chess.engine

In [ ]:
board = chess.variant.AntichessBoard("r2qk2r/p2p3p/8/8/8/8/P2P3P/R2QK2R w - - 0 1")

In [ ]:
board

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci("C:/Users/jerne/Downloads/fairy-stockfish-largeboard_x86-64.exe")
possible_moves = get_move_evaluation(engine, board, time_per_move=0.1)

In [ ]:
possible_moves

In [ ]:
pd.read_csv()

In [ ]:
board = chess.variant.AntichessBoard("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w - - 0 1")

In [ ]:
board